In [1]:
import pandas as pd
import gmaps
from ast import literal_eval

from python_scripts.render_closures import render_closures
from python_scripts.intersect_to_coords import intersect_to_coords
from python_scripts.format_here_data import format_here_data
import python_scripts.config as config

gmaps.configure(api_key= config.gmaps_key)

# The "Finished" Product

In this notebook I will render a map of road closures including data from bother here.com and Twitter's API, to include in our presentation and to show the workflow of the map making process.

## Read and Format Data

In [2]:
# read in the data
here_data = pd.read_csv('../data/2-interim/here_maps_output_Houston, TX_2019-01-17-12PM')
tweets = pd.read_csv('../data/3-final/output_tweets')

In [3]:
# tweets format wasn't quite right. Let's try this:
tweets.rename(index=str, columns={"tweet": "text"},inplace=True)
tweets

,city,start,end,text
0,"Houston, TX",288 S Fwy NB & The S Lp,288 S Fwy NB & Reed Rd.,Closed due to road construction on 288 S Fwy N...
1,"Houston, TX",S Sam Houston Tollway EB & Hwy 288,S Sam Houston Tollway EB & Cullen,Closed due to road construction in #Southside ...


In [4]:
# format the here.com data
here_data = format_here_data(here_data)
here_data

,start,end,text
0,"(29.6307, -95.16875)","(29.630614, -95.171866)",None
1,"(29.7369, -95.34869)","(29.73713, -95.34964)",None
2,"(29.92321, -95.62916)","(29.92392, -95.62903)",None
3,"(29.630513, -95.171997)","(29.63044, -95.16984)",None
4,"(29.69674, -95.29231)","(29.69593, -95.29787)",None
5,"(29.760211, -95.35475)","(29.76199, -95.35303)",None
6,"(29.89392, -95.5892)","(29.89392, -95.5892)",None
7,"(29.729192, -95.462069)","(29.726718, -95.467795)",None
8,"(29.718169, -95.460046)","(29.721648, -95.459828)",None


In [5]:
# geocode the Twitter intersections, append the closures to here_data
# Google Maps API doesn't allow more than 10 closures per map
all_closures = intersect_to_coords(tweets,here_data)[:11]

# I'll manually alter the two labels we have. Future versions of the project could use a
# model to extract the relevant label data
all_closures.text[:2] = 'road construction'
all_closures

,start,end,text
0,"(29.7421856, -95.36452899999999)","(29.4867858, -95.39786819999999)",road construction
1,"(29.66714289999999, -95.5605762)","(29.59933909999999, -95.35629120000002)",road construction
2,"(29.6307, -95.16875)","(29.630614, -95.171866)",None
3,"(29.7369, -95.34869)","(29.73713, -95.34964)",None
4,"(29.92321, -95.62916)","(29.92392, -95.62903)",None
5,"(29.630513, -95.171997)","(29.63044, -95.16984)",None
6,"(29.69674, -95.29231)","(29.69593, -95.29787)",None
7,"(29.760211, -95.35475)","(29.76199, -95.35303)",None
8,"(29.89392, -95.5892)","(29.89392, -95.5892)",None
9,"(29.729192, -95.462069)","(29.726718, -95.467795)",None


In [6]:
all_closures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
start    11 non-null object
end      11 non-null object
text     2 non-null object
dtypes: object(3)
memory usage: 344.0+ bytes


In [7]:
# One row's start and end points were the same, which throws errors
all_closures.drop([0,8],inplace=True)
all_closures.reset_index(drop=True,inplace=True)
all_closures

,start,end,text
0,"(29.66714289999999, -95.5605762)","(29.59933909999999, -95.35629120000002)",road construction
1,"(29.6307, -95.16875)","(29.630614, -95.171866)",None
2,"(29.7369, -95.34869)","(29.73713, -95.34964)",None
3,"(29.92321, -95.62916)","(29.92392, -95.62903)",None
4,"(29.630513, -95.171997)","(29.63044, -95.16984)",None
5,"(29.69674, -95.29231)","(29.69593, -95.29787)",None
6,"(29.760211, -95.35475)","(29.76199, -95.35303)",None
7,"(29.729192, -95.462069)","(29.726718, -95.467795)",None
8,"(29.718169, -95.460046)","(29.721648, -95.459828)",None


## Render Map

In [8]:
# Now we render the road closures using Google Maps
render_closures(all_closures)

Figure(layout=FigureLayout(height='400px', margin='0 auto 0 auto', padding='1px', width='700px'))

## Export Final DataFrame

For thoroughness's sake.

In [10]:
all_closures.to_csv('../data/3-final/final-closure-df.csv')